TODO:
- Figure out how to properly implement rebalancing
- Add user input reading to determine parameters

Imports

In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import seaborn as sns
import math

Parameters

In [156]:
# Tickers of ETFS and their portfolio weighings
products = {
          "SPY":0.6,    #SP500
          "QQQ":0.0,   #NASDAQ100
          "IWM":0.0,    #Russel2000
           #Bonds
          "BIL":0.0,     #0-3 month 
          "SHY":0.4,    #1-3 yr 
          "IEF":0.0,      #7-10yr
          "TLT":0.0       #20+yr
}
# Start and end dates for the test 
start_date = '2010-01-01'
end_date = '2020-01-01'

# Deposit frequency and amount
 # ["daily","weekly","monthly","quarterly","yearly"] -> [1,7,30,120,365]
frequency = 7  # Deposit frequency 
deposit = 100 # Deposit amount
  
# Rebalancing?
rebalancing = False
rebalancing_frequency = 120

Build the dataframe

In [157]:
def create_df(products,start_date,end_date,frequency,deposit,rebalancing,rebalancing_frequency):
    # Get data from yahoo finance
    tickers_use = sorted([key for key,value in products.items() if value != 0])
    #dates = pd.date_range(start_date, end_date,frequency='B')

    data = yf.download(tickers_use, start=start_date, end=end_date)
    df = pd.DataFrame(data['Adj Close'])
    df.reset_index(inplace=True)

    # Add Transact, Deposit, Rebalancing Flags
    df['Deposit'] = 0
    df['Transact'] = False
    df['Rebalance'] = False


    for i in range(0,len(df),frequency):
        df.at[i,'Transact'] = True
        df.at[i,'Deposit'] = deposit

    for i in range(rebalancing_frequency,len(df),rebalancing_frequency):
        df.at[i,'Rebalance'] = True


    #Calculate amount of each instrument bought on given date
    units_bought = [f'{ticker}_units_bought' for ticker in tickers_use ]
    units_cum = [f'{ticker}_units_cum' for ticker in tickers_use ]
    units_value = [f'{ticker}_value' for ticker in tickers_use ]

    for i in range(len(units_bought)):
        p = tickers_use[i]
        weight = products[p] 
        df[units_bought[i]] =  df.Deposit * weight / df[tickers_use[i]]

        df[units_cum[i]] = df[units_bought[i]].cumsum()

        weight = products[tickers_use[i]] 
        df[units_value[i]] =  df[tickers_use[i]] * df[units_cum[i]]
        
        #if df['Rebalance'][i] == True: ## Commence rebalancing 

    #Portfolio value and cumulative deposits
    df['Portfolio_Value'] = df[units_value].sum(axis=1)
    df['Deposits_cum'] = df.Deposit.cumsum()
    df['OverallPnL'] = (df['Portfolio_Value'] - df['Deposits_cum'])/ df['Deposits_cum']
    #Figure out smart way to do this
    df['DoDPnL'] = 0
    for i in range(1,len(df)):
        df.at[i,'DoDPnL'] = (df.at[i,'Portfolio_Value'] - df.at[i-1,'Portfolio_Value']-df.at[i,'Deposit'])/df.at[i-1,'Portfolio_Value']
            
    
    return df
    

In [158]:
df = create_df(products,start_date,end_date,frequency,deposit,rebalancing,rebalancing_frequency)

[*********************100%***********************]  2 of 2 completed


In [159]:
df

,Date,SHY,SPY,Deposit,Transact,Rebalance,SHY_units_bought,SHY_units_cum,SHY_value,SPY_units_bought,SPY_units_cum,SPY_value,Portfolio_Value,Deposits_cum,OverallPnL,DoDPnL
0,2010-01-04,74.582443,88.860382,100,True,False,0.536319,0.536319,40.000000,0.675217,0.675217,60.000000,100.000000,100,0.000000,0.000000
1,2010-01-05,74.672195,89.095604,0,False,False,0.000000,0.536319,40.048136,0.000000,0.675217,60.158826,100.206962,100,0.002070,0.002070
2,2010-01-06,74.681175,89.158340,0,False,False,0.000000,0.536319,40.052952,0.000000,0.675217,60.201186,100.254138,100,0.002541,0.000471
3,2010-01-07,74.663223,89.534683,0,False,False,0.000000,0.536319,40.043324,0.000000,0.675217,60.455299,100.498623,100,0.004986,0.002439
4,2010-01-08,74.735016,89.832664,0,False,False,0.000000,0.536319,40.081828,0.000000,0.675217,60.656501,100.738329,100,0.007383,0.002385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,2019-12-24,82.938919,307.631134,0,False,False,0.000000,183.482383,15217.830532,0.000000,142.526134,43845.476323,59063.306855,35900,0.645217,0.000054
2512,2019-12-26,82.968384,309.268707,0,False,False,0.000000,183.482383,15223.236789,0.000000,142.526134,44078.873307,59302.110096,35900,0.651869,0.004043
2513,2019-12-27,83.027214,309.192139,100,True,False,0.481770,183.964153,15274.031106,0.194054,142.720188,44127.960280,59401.991385,36000,0.650055,-0.000002
2514,2019-12-30,83.056671,307.487457,0,False,False,0.000000,183.964153,15279.450155,0.000000,142.720188,43884.667830,59164.117984,36000,0.643448,-0.004004


Plotting and stuff